# 🚀 Advanced Coexistence Simulation Framework using ms-van3t & Sionna-RT

This notebook provides a complete pipeline for simulating and analyzing coexistence between different vehicular communication technologies (e.g., DSRC, C-V2X, NR-V2X) in realistic urban scenarios.

Key features:
- Environment setup (cloning repositories and building Sionna-RT Docker image)
- Post-processing script for SINR, interference, and RSSI calculation
- High-quality SINR PDF visualization
- Disagreement Ratio (DR) metric for comparing simulators
- JSON schemas for communication with Sionna-RT ray tracer
- Detailed simulation loop pseudocode

**Note**: This is currently a post-processing and visualization toolkit built around simulated packet data. Full end-to-end simulation requires additional integration layers (see TODO section below).

In [ ]:
# Environment Setup: Clone repositories and build Sionna-RT Docker image
!git clone https://github.com/ms-van3t-devs/ms-van3t ms-van3t
%cd ms-van3t

!git clone https://github.com/nvlabs/sionna sionna-rt

# Build Docker image for Sionna-RT (GPU-accelerated ray tracing)
!docker build -t sionna-rt:epic-edition sionna-rt/

print("\nSetup complete! Ready for vehicular communication simulations.")

In [ ]:
# Coexistence Analysis Script
# This script processes simulated packet traces and computes per-packet SINR, RSSI, and interference

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde

# Physical constants
K_B = 1.380649e-23  # Boltzmann constant (J/K)
T = 290.0           # Standard noise temperature (Kelvin)

# Power conversion utilities
def dbm_to_watts(dbm):
    """Convert power from dBm to Watts."""
    return 10 ** ((dbm - 30) / 10.0)

def watts_to_dbm(watts):
    """Convert power from Watts to dBm. Returns -inf for zero or negative input."""
    return 10 * np.log10(watts) + 30 if watts > 0 else -np.inf

# Load packet trace from simulation (expected columns: time, rx_id, tx_id, tx_power_dBm, path_gain_dB, freq_start_Hz, freq_end_Hz, technology, ...)
df = pd.read_csv("sim_packets.csv", parse_dates=['time'])
print(f"Loaded {len(df)} packet records.")

# Check if frequencies overlap (simple interval overlap)
def check_resource_overlap(tx_row, other_row):
    """Return True if two transmissions overlap in frequency."""
    a_start, a_end = tx_row['freq_start_Hz'], tx_row['freq_end_Hz']
    b_start, b_end = other_row['freq_start_Hz'], other_row['freq_end_Hz']
    return not (a_end <= b_start or b_end <= a_start)

# Group packets by reception time and receiver ID (all concurrent receptions at one RX)
grouped = df.groupby(['time', 'rx_id'])

results = []
GTHR_DB = -120                  # Minimum path gain threshold (dB) to consider an interferer
GTHR_LIN = 10 ** (GTHR_DB / 10.0)

for (time, rx), group in grouped:
    group = group.copy()
    # Convert path gain to linear scale
    group['path_gain_lin'] = 10 ** (group['path_gain_dB'] / 10.0)
    # Convert TX power to Watts and compute received power
    group['tx_power_W'] = group['tx_power_dBm'].apply(dbm_to_watts)
    group['rx_power_W'] = group['tx_power_W'] * group['path_gain_lin']

    for idx, desired in group.iterrows():
        # All other potential interferers at the same RX and time
        interferers = group.loc[group.index != idx]
        
        # Keep only those overlapping in frequency
        interferers = interferers[interferers.apply(lambda r: check_resource_overlap(desired, r), axis=1)]
        
        # Apply path gain threshold (ignore very weak signals)
        interferers = interferers[interferers['path_gain_lin'] >= GTHR_LIN]
        
        # Total interference power
        pint_w = interferers['rx_power_W'].sum() if not interferers.empty else 0.0

        # Bandwidth and thermal noise
        bw = desired['freq_end_Hz'] - desired['freq_start_Hz']
        n0_w = K_B * T * bw
        
        # SINR calculation
        sinr_lin = desired['rx_power_W'] / (n0_w + pint_w + 1e-30)
        sinr_db = 10 * np.log10(sinr_lin)
        
        rssi_dbm = watts_to_dbm(desired['rx_power_W'])

        results.append({
            'time': time,
            'rx_id': rx,
            'tx_id': desired['tx_id'],
            'technology': desired['technology'],
            'SINR_dB': sinr_db,
            'PRx_dBm_est': rssi_dbm,
            'Pint_dBm': watts_to_dbm(pint_w)
        })

# Save detailed per-packet results
res_df = pd.DataFrame(results)
res_df.to_csv("sim_results_sinr.csv", index=False)
print(f"Saved {len(res_df)} SINR records to sim_results_sinr.csv")

# SINR Distribution Visualization (PDF with KDE)
plt.figure(figsize=(10, 6))
sns.set(style="whitegrid")
for tech, g in res_df.groupby('technology'):
    vals = g['SINR_dB'].dropna().values
    if len(vals) > 1:  # Need at least 2 points for KDE
        kde = gaussian_kde(vals)
        x_grid = np.linspace(vals.min(), vals.max(), 1000)
        plt.plot(x_grid, kde(x_grid), label=f"{tech}", linewidth=2)
        plt.fill_between(x_grid, kde(x_grid), alpha=0.3)

plt.xlabel("SINR (dB)")
plt.ylabel("Probability Density")
plt.title("SINR Distribution (Simulated)")
plt.legend()
plt.grid(True, linestyle='--')
plt.savefig("sinr_pdf_epic.png", dpi=300, bbox_inches='tight')
plt.close()
print("SINR PDF plot saved as sinr_pdf_epic.png")

# Disagreement Ratio between two decision sets (e.g., different simulators)
def compute_disagreement_ratio(df_a, df_b):
    """Compute fraction of packets where verdict differs between two simulators."""
    key_cols = ['time', 'tx_id', 'rx_id']
    a = df_a.set_index(key_cols)
    b = df_b.set_index(key_cols)
    union_idx = a.index.union(b.index)
    a_vals = a.reindex(union_idx)['verdict'].fillna(-1).astype(int)
    b_vals = b.reindex(union_idx)['verdict'].fillna(-1).astype(int)
    return (a_vals != b_vals).sum() / len(union_idx)

# Example usage (uncomment when files are available):
# df_a = pd.read_csv("decisions_msvan3t.csv", parse_dates=['time'])
# df_b = pd.read_csv("decisions_van3twin.csv", parse_dates=['time'])
# print(f"Disagreement Ratio: {compute_disagreement_ratio(df_a, df_b):.4f}")

## ⚠️ Current Limitations & TODO List

This notebook is a strong post-processing and visualization framework, but it is not yet a complete end-to-end simulator. The following components are missing or incomplete:

1. **Real-time communication with Sionna-RT**  
   → No actual socket/gRPC client to send `chan_req` and receive `chan_resp`. Currently only JSON schemas are defined.

2. **Integration with mobility simulator (SUMO)**  
   → No code to load SUMO traces, generate vehicle positions, or drive location updates.

3. **Accurate resource grid modeling**  
   → Current overlap check is basic frequency range only. Real C-V2X/NR-V2X needs subchannel/PRB-level resource selection.

4. **Packet reception verdict (BLER mapping)**  
   → SINR is computed but not mapped to Block Error Rate (BLER) curves to determine packet success/failure.

5. **Network-level performance metrics**  
   → Missing PDR, throughput, latency, fairness, awareness range, etc.

6. **Multi-technology coexistence logic**  
   → Technology field exists but no specific resource selection or sensing procedures for DSRC vs C-V2X vs NR.

7. **Performance optimization**  
   → Current loop can be slow on large traces; consider vectorization or Numba acceleration.

8. **Advanced visualization**  
   → Add CDF plots, spatial heatmaps, animation of vehicle positions and links.

9. **Configuration management**  
   → All parameters are hard-coded. Add config file (YAML/JSON) or argparse support.

**Next steps**: Prioritize implementing the Sionna-RT client and SUMO integration to enable full closed-loop simulation.

## 📡 JSON Communication Schemas with Sionna-RT

These messages define the interface between the network simulator and the ray-tracing engine.

In [ ]:
# Scene Initialization Message
{
  "type": "init",
  "scene_id": "turin_center",
  "meshes": [
    {"id": "bldg_1", "path": "/meshes/bldg_1.obj", "material": {"eps_r": 5.31, "sigma": 0.4838}},
    # Additional buildings, roads, vegetation, etc.
  ],
  "vehicles": [
    {"veh_id": "veh_12", "mesh_id": "car_sedan", "antenna_offset": [0.0, 0.0, 1.8]}
  ],
  "timestamp": 0
}

In [ ]:
# Location Update Message (sent when vehicle moves significantly)
{
  "type": "loc_update",
  "vehicle_id": "veh_12",
  "pos": [x, y, z],
  "vel": [vx, vy, vz],
  "heading": theta,
  "timestamp": t
}

In [ ]:
# Channel Request
{
  "type": "chan_req",
  "tx_id": "veh_A",
  "rx_id": "veh_B",
  "timestamp": t
}

# Channel Response
{
  "type": "chan_resp",
  "tx_id": "veh_A",
  "rx_id": "veh_B",
  "G_dB": -84.3,
  "tau_s": 1.2e-6,
  "los_flag": 0,
  "paths": [
    {"alpha": "complex", "delay": 1e-9, "doppler": 120.0, "AOD": [az, el], "AOA": [az, el]}
  ]
}

## 🔄 Simulation Loop Pseudocode

High-level structure of a complete event-driven simulator.

In [ ]:
# Initialize scene in ray tracer
send_init_to_rt(scene_definition)

# Main simulation loop
while simulation_running:
    # Advance mobility (e.g., from SUMO trace)
    update_vehicle_positions()
    
    # Update ray tracer only if significant movement
    if significant_position_change():
        send_location_updates()
    
    # For each packet transmission event
    for packet in scheduled_packets:
        # Request channel from ray tracer (or use cache)
        channel = get_channel(tx=packet.tx, rx=packet.rx)
        
        # Find concurrent interferers on overlapping resources
        interferers = find_concurrent_interferers(packet)
        
        # Compute SINR per resource block
        sinr = compute_sinr(desired_power, interference_power, noise_power)
        
        # Determine packet success using BLER curve
        verdict = map_sinr_to_bler(sinr) < random_draw()
        
        # Record all metrics
        log_metrics(packet, sinr, verdict)

print("Simulation finished.")

# ✅ Summary

You now have a polished, well-documented notebook with English comments throughout, clear visualizations, and an explicit TODO section highlighting current limitations and recommended next steps.

Feel free to extend any section—especially the integration layers—to turn this into a full-featured research-grade coexistence simulator!